In [ ]:
import pandas as pd
import glob
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split # module to split our data into train and test sets
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
import statsmodels.tools

In [ ]:
#find the path of all the files of the datasets
file_directory = r'/Users/yanhuijun/DigitalFutures/Poject2-Car'

In [ ]:
file_path = os.path.join(file_directory , "*.txt")

In [ ]:
all_files = glob.glob(file_path)

In [ ]:
# build a dataframe and loop the files in the dataframe
dataframes = []

for filename in all_files:
    df = pd.read_csv(filename)
    df['filename'] = filename
    dataframes.append(df)

#concat the dataframe
cars = pd.concat(dataframes, axis=0, ignore_index=True)

In [ ]:
# rename the complicated column name
cars['car model'] = cars['filename'].str.replace('/Users/yanhuijun/DigitalFutures/Poject2-Car/', '', regex=False).str.replace('.txt', '',regex=False)

## 1. Building New dataframe

In [ ]:
cars.columns

Index(['name', 'link', 'price', 'mileage', 'BHP', 'transmission', 'fuel',
       'owners', 'body', 'ULEZ', 'engine', 'year', 'filename', 'car model'],
      dtype='object')

In [ ]:
cars_new = cars[['car model','link','price','mileage', 'BHP', 'transmission', 'fuel',
                 'owners', 'body', 'ULEZ', 'engine', 'year']].copy()

In [ ]:
# as the 'name' is different, we dropped it to clean the new dataframe (drop the duplication)
cars_new.drop_duplicates(keep = 'last', inplace = True)

## 2.EDA


#### Cleaning special characters

In [ ]:
#clean special characters
cars_new['price'] = cars_new['price'].str.replace('£','').str.replace(',','')

In [ ]:
cars_new['year'] = cars_new['year'].str.replace(' (.*)','')

<ipython-input-11-6549f55273a9>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  cars_new['year'] = cars_new['year'].str.replace(' (.*)','')


In [ ]:
cars_new['engine'] = cars_new['engine'].str.replace('L','')

In [ ]:
cars_new.head()

,car model,link,price,mileage,BHP,transmission,fuel,owners,body,ULEZ,engine,year
0,Volkswagen_Amarok,https://www.autotrader.co.uk/car-details/20220...,9960,113000.0,161.0,Manual,Diesel,3.0,Pickup,NaN,2.0,2011
1,Volkswagen_Amarok,https://www.autotrader.co.uk/car-details/20220...,13995,50767.0,161.0,Manual,Diesel,NaN,Pickup,NaN,2.0,2011
2,Volkswagen_Amarok,https://www.autotrader.co.uk/car-details/20220...,11500,112000.0,161.0,Manual,Diesel,3.0,Pickup,NaN,2.0,2011
3,Volkswagen_Amarok,https://www.autotrader.co.uk/car-details/20220...,12950,128848.0,161.0,Manual,Diesel,4.0,Pickup,NaN,2.0,2011
4,Volkswagen_Amarok,https://www.autotrader.co.uk/car-details/20220...,12495,92000.0,161.0,Manual,Diesel,NaN,Pickup,NaN,2.0,2011


#### Drop null which the percentage is below 2.6%

In [ ]:
# building a function which can see the number of null value in each columns and it's percentage.
def null_vals(dataframe):

    null_vals = dataframe.isnull().sum()
    total_cnt = len(dataframe)
    null_vals = pd.DataFrame(null_vals,columns=['null'])
    null_vals['percent'] = round((null_vals['null']/total_cnt)*100,3)
    return null_vals.sort_values('percent', ascending=False)

null_vals(cars_new)

,null,percent
owners,19040,68.470
BHP,14870,53.474
ULEZ,4408,15.852
body,701,2.521
year,616,2.215
engine,118,0.424
transmission,14,0.050
fuel,6,0.022
mileage,5,0.018
car model,0,0.000


In [ ]:
#drop null in the columns which the percentage is below 2.6%
cars_new.dropna(subset = ['body','year','engine','transmission','fuel','mileage'], inplace = True)

In [ ]:
null_vals(cars_new)

,null,percent
owners,18041,68.345
BHP,13912,52.703
ULEZ,3956,14.987
car model,0,0.000
link,0,0.000
price,0,0.000
mileage,0,0.000
transmission,0,0.000
fuel,0,0.000
body,0,0.000


#### Drop extreme data

In [ ]:
# found probably wrong data and drop it
cars_new['BHP'].max()
#drop the extreme value
cars_new = cars_new.drop(cars_new.index[cars_new['BHP'] == cars_new['BHP'].max()])

In [ ]:
# other null value will be filled in in the feature engineering process.
null_vals(cars_new)

,null,percent
owners,18041,68.347
BHP,13912,52.705
ULEZ,3956,14.987
car model,0,0.000
link,0,0.000
price,0,0.000
mileage,0,0.000
transmission,0,0.000
fuel,0,0.000
body,0,0.000


#### Exploration

In [ ]:
# check the number of null value and data type
cars_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26396 entries, 0 to 27833
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   car model     26396 non-null  object 
 1   link          26396 non-null  object 
 2   price         26396 non-null  object 
 3   mileage       26396 non-null  float64
 4   BHP           12484 non-null  float64
 5   transmission  26396 non-null  object 
 6   fuel          26396 non-null  object 
 7   owners        8355 non-null   float64
 8   body          26396 non-null  object 
 9   ULEZ          22440 non-null  object 
 10  engine        26396 non-null  object 
 11  year          26396 non-null  object 
dtypes: float64(3), object(9)
memory usage: 2.6+ MB


In [ ]:
# transmission has two categories, so it can be converted to category
cars_new['transmission'].value_counts()

Manual       16667
Automatic     9729
Name: transmission, dtype: int64

In [ ]:
#body can also convert to category to encode
cars_new['body'].value_counts()

Hatchback      16041
SUV             7108
Estate          1336
Saloon           766
MPV              510
Pickup           441
Convertible      194
Name: body, dtype: int64

In [ ]:
#ULEZ can also convert to category to encode (fill null value later)
cars_new['ULEZ'].value_counts()

ULEZ    22440
Name: ULEZ, dtype: int64

In [ ]:
# although owners is numeric, we considered to convert it to category
cars_new['owners'].value_counts()

2.0     4986
3.0     1959
4.0      827
5.0      405
6.0       90
7.0       45
8.0       24
9.0        9
10.0       5
11.0       2
14.0       1
15.0       1
13.0       1
Name: owners, dtype: int64

In [ ]:
# we dropped the reg value as we decided to categorise it as different years
cars_new['year'].value_counts()

2019    4224
2018    3977
2017    2733
2020    2235
2016    2185
2021    2181
2015    1864
2014    1419
2013    1235
2012    1019
2011     950
2010     634
2009     596
2008     427
2007     332
2006     231
2005     154
Name: year, dtype: int64

In [ ]:
# as there are not enough numbers for some types of engine, we decided to bin it to different groups in the feature engineering process.
cars_new['engine'].value_counts()

2.0    9360
1.0    5681
1.5    3104
1.4    2588
1.6    2519
1.2    1610
3.0    1045
1.8     187
1.9     145
3.2     108
2.5      31
3.6      11
5.0       3
4.2       3
6.0       1
Name: engine, dtype: int64

In [ ]:
# Saves the new and improved cleaned cars data
cars_new.to_csv("cars_new.csv", index=False)